In [48]:
# 导入依赖包
import requests as req
import json
import re
from bs4 import BeautifulSoup
from lxml import etree

In [49]:
# 发送请求

# 伪装
heads ={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36 Edg/141.0.0.0',
    'Cookie' : 'll="118302"; bid=GmUemmMKVUA; _pk_id.100001.4cf6=4ebba9ce5ff2da0a.1757987431.; __utmz=30149280.1757987432.1.1.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; __utmz=223695111.1757987432.1.1.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; __yadk_uid=zUwiroJynzpJtsayAHZ1SNSr7ILPBCt2; _vwo_uuid_v2=D8D6011DB1CFB180005FA9A7EB6C1E29D|9f8059e35747060f1b2beb84cb0039e7; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1758067647%2C%22https%3A%2F%2Fwww.baidu.com%2Flink%3Furl%3DN369YYCCjQz7Haokiak4MK6ozhTmmmN2zvL3axLYudLSI59upGbF5amcLwGdrjYD%26wd%3D%26eqid%3D84073e5b02607de20000000668c8c262%22%5D; ap_v=0,6.0; __utma=30149280.1858202652.1757987432.1758031158.1758067662.3; __utmc=30149280; __utma=223695111.1249666086.1757987432.1758031158.1758067662.3; __utmc=223695111'
}

# 请求
res = req.get('https://movie.douban.com/', headers = heads)

# 中文编码UTF-8
res.encoding = 'utf-8'

# # 打印信息
# print(res.text)

In [ ]:
# 使用xpath解析数据
# 把html文本转成一棵树
htree = etree.HTML(res.text)
# //*[@id="screening"]/div[2]/ul/li[1]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[2]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[3]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[4]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[5]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[6]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[7]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[8]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[9]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[10]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[11]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[12]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[13]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[14]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[15]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li[16]/ul/li[2]/a
# //*[@id="screening"]/div[2]/ul/li
movie_items = htree.xpath('//*[@id="screening"]/div[2]/ul/li')
# 使用xpath表达式解析数据
# 遍历每个电影条目 
for index, item in enumerate(movie_items, start=1): 
    # 在当前的item中，使用相对路径获取电影名称 
    # 注意：这里使用相对路径，以当前item为起点 
    name_list = item.xpath('.//ul/li[2]/a/text()') 
    # 因为text()返回的是列表，我们需要检查是否有数据 
    if name_list: 
        movie_name = name_list[0].strip() 
    else: 
        movie_name = "暂无名称" 
 
    # 获取评分 
    rating_list = item.xpath('.//ul/li[3]/span[2]/text()') 
    if rating_list: 
        movie_rating = rating_list[0].strip() 
    else: 
        movie_rating = "暂无评分" 
 
    # 打印结果，可以按照需要调整格式 
    print(f"电影{index}: 名称: {movie_name}, 评分: {movie_rating}") 

In [ ]:
终止行

In [ ]:
#解析电影名称
#<li class="ui-slide-item"  data-title="(.*?)" data-release=
# alt="(.*?)" rel="nofollow"/>
result1 = re.findall('alt="(.*?)" rel="nofollow"/>', res.text)
print(result1)

In [ ]:
#解析评分
#</span><span class="subject-rate">(.*?)</span>
# ">(.*?)</span>
#  data-rate="(.*?)" 
result2 = re.findall(' data-rate="(.*?)" ', res.text)
print(result2)

In [ ]:
# 打印信息
startIndex = res.text.index('<ul class="ui-slide-content" data-slide-index="1" data-index-max="9">')
endIndex = res.text.index(' <li class="ui-slide-item">')
print(startIndex)
print(endIndex)

In [ ]:
allNewsStr = res.text[startIndex:endIndex].replace('<ul class="ui-slide-content" data-slide-index="1" data-index-max="9">', '')
print(allNewsStr)

In [ ]:
allNewsStr = res.text[startIndex:endIndex].replace('<ul class="ui-slide-content" data-slide-index="1" data-index-max="9">', '')
print(allNewsStr)
# 把字符串转成字典格式
news = json.loads(allNewsStr)
print(news['hotsearch'])

In [ ]:
# 解析HTML
soup = BeautifulSoup(allNewsStr, 'html.parser')

# 查找所有电影条目（class包含"ui-slide-item"的<li>标签）
movie_items = soup.find_all('li', class_=lambda x: x and 'ui-slide-item' in x)

# 存储结果的列表
movies = []

# 循环处理每个电影条目
for item in movie_items:
    # 提取电影名称 - 方式1：从标题元素获取
    title_tag = item.find('li', class_='title')
    if title_tag:
        movie_name = title_tag.a.text.strip()
    else:
        # 方式2：从data-title属性获取（备用方式）
        movie_name = item.get('data-title', '未知电影')
    
    # 提取评分
    rating = soup.find('span', class_='subject-rate').text.strip()
    
    # 添加到结果列表
    movies.append({
        'name': movie_name,
        'rating': rating
    })

# 打印结果
for i, movie in enumerate(movies, 1):
    print(f"{i}. 电影名称: {movie['name']}, 评分: {movie['rating']}")

In [ ]:
# # 提取电影名称
# movie_name = soup.find('li', class_='title').a.text.strip()  # 方式1：从标题元素获取
# # 提取评分
# rating = soup.find('span', class_='subject-rate').text.strip()

In [ ]:
# print(f"电影名称: {movie_name}")
# print(f"豆瓣评分: {rating}")

In [ ]:
# 将数据写入move.txt文件
with open('movie.txt', 'w', encoding='utf-8') as f:
    for movie in movies:
        f.write(f"电影名称: {movie['name']}, 评分: {movie['rating']}\n") 